In [2]:
import pandas as pd

In [3]:
# read in data
movie_start_df = pd.read_csv("clean_upcoming_df_2022.csv")
movie_start_df

,movie_id,title,movie_rating,release_date,run_time,genre_name,director,writer,actors,language,meta_score,imdb_rating,box_office,Trash
0,0,The Batman,PG-13,2022-03-04,175,"Action, Crime, Drama",Matt Reeves,"Matt Reeves, Peter Craig, Bob Kane","Zoë Kravitz, Robert Pattinson, Barry Keoghan",English,0.0,0.0,0.0,min
1,1,A Day to Die,0,1970-01-01,0,Action,Wes Miller,"Rab Berry, Scott Mallace","Bruce Willis, Frank Grillo, Kevin Dillon",English,0.0,0.0,0.0,NaN
2,2,He Was Asking for It: A Noir Film,0,2022-04-30,0,"Short, Crime",Rocky Karlage,"Rocky Karlage, Staci Layne Wilson","Rocky Karlage, Patrick Lamba, Charlie Jiin",English,0.0,0.0,0.0,NaN
3,3,Tyson's Run,PG,2022-03-11,103,Drama,Kim Bass,Kim Bass,"Major Dodson, Amy Smart, Barkhad Abdi",English,0.0,0.0,0.0,min
4,4,X,R,2022-03-18,0,Horror,Ti West,Ti West,"Jenna Ortega, Mia Goth, Brittany Snow",English,0.0,0.0,0.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78,78,Aquaman and the Lost Kingdom,0,2022-12-16,0,"Action, Adventure, Fantasy",James Wan,"David Leslie Johnson-McGoldrick, Paul Norris, ...","Temuera Morrison, Nicole Kidman, Jason Momoa",English,0.0,0.0,0.0,NaN
79,79,Untitled Illumination Entertainment Project,0,2022-12-21,0,"Animation, Adventure, Comedy","Aaron Horvath, Michael Jelenic","Matthew Fogel, Shigeru Miyamoto","Anya Taylor-Joy, Chris Pratt, Seth Rogen",English,0.0,0.0,0.0,NaN
80,80,I Wanna Dance with Somebody,0,2022-12-23,0,"Biography, Music",Kasi Lemmons,Anthony McCarten,"Stanley Tucci, Tamara Tunie, Clarke Peters",0,0.0,0.0,0.0,NaN
81,81,A Man Called Otto,0,1970-01-01,0,"Comedy, Drama",Marc Forster,Fredrik Backman,"Tom Hanks, Rachel Keller, Manuel Garcia-Rulfo",English,0.0,0.0,0.0,NaN


In [4]:
# create dfs for tables
role_df = pd.DataFrame({"role_id": [0,1,2], "role": ["Director", "Writer", "Actor"]})
person_df = pd.DataFrame({"person_id": [], "person": []})
movie_role_person_df = pd.DataFrame(columns=["movie", "person", "role"])

In [5]:
# set indexes
role_df.set_index(['role_id'], inplace=True)
person_df.set_index(["person_id"], inplace=True)
role_df

,role
role_id,
0,Director
1,Writer
2,Actor


In [6]:
person_df.head()

,person
person_id,


In [7]:
# iterate through movie_df
for index, row in movie_start_df.iterrows():
    # split out all people with roles
    directors = row["director"].split(", ")
    writers = row["writer"].split(", ")
    actors = row["actors"].split(", ")


    for director in directors:
        # check if person in person_df
        if director not in person_df["person"].values:
            new_person_row = {"person": director}
            person_df = person_df.append(new_person_row, ignore_index=True)
        # add new junction row
        new_junction_row = {"movie": row["movie_id"], "role": 0, "person": person_df.index[person_df["person"] == director].tolist()}
        movie_role_person_df = movie_role_person_df.append(new_junction_row, ignore_index=True)
        
    for writer in writers:
        # check if person in person_df
        if writer not in person_df["person"].values:
            new_person_row = {"person": writer}
            person_df = person_df.append(new_person_row, ignore_index=True)
        # add new junction row
        new_junction_row = {"movie": row["movie_id"], "role": 1, "person": person_df.index[person_df["person"] == writer].tolist()}
        movie_role_person_df = movie_role_person_df.append(new_junction_row, ignore_index=True)

    for actor in actors:
        # check if person in person_df
        if actor not in person_df["person"].values:
            new_person_row = {"person": actor}
            person_df = person_df.append(new_person_row, ignore_index=True)
        # add new junction row
        new_junction_row = {"movie": row["movie_id"], "role": 2, "person": person_df.index[person_df["person"] == actor].tolist()}
        movie_role_person_df = movie_role_person_df.append(new_junction_row, ignore_index=True)

/var/folders/ff/04lnt98n62z2m6h1qpf6gglr0000gn/T/ipykernel_92127/3157368159.py:11: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if director not in person_df["person"].values:


In [8]:
person2 = person_df.rename(columns={'person':'name'})
person2.index.name = "person_id"
person2.head()

,name
person_id,
0,Matt Reeves
1,Peter Craig
2,Bob Kane
3,Zoë Kravitz
4,Robert Pattinson


In [9]:
# converts person column from list type to string type
movie_role_person_df['person']  = [','.join(map(str, row)) for row in movie_role_person_df['person']]

In [10]:
movie_role_person_df2 = movie_role_person_df.rename(columns={'movie':'movie_id', 'person':'person_id', 'role':'role_id'})
movie_role_person_df2

,movie_id,person_id,role_id
0,0,0,0
1,0,0,1
2,0,1,1
3,0,2,1
4,0,3,2
...,...,...,...
499,82,424,0
500,82,424,1
501,82,59,2
502,82,372,2


In [11]:
role_df.to_csv("upcoming_role_table.csv")
person2.to_csv("upcoming_person_table.csv")
movie_role_person_df2.to_csv("upcoming_movie_role_person_junction_table.csv", index=False)

In [12]:
movie_role_person_df2

,movie_id,person_id,role_id
0,0,0,0
1,0,0,1
2,0,1,1
3,0,2,1
4,0,3,2
...,...,...,...
499,82,424,0
500,82,424,1
501,82,59,2
502,82,372,2


In [13]:
person2

,name
person_id,
0,Matt Reeves
1,Peter Craig
2,Bob Kane
3,Zoë Kravitz
4,Robert Pattinson
...,...
420,Fredrik Backman
421,Tom Hanks
422,Rachel Keller


In [14]:
role_df

,role
role_id,
0,Director
1,Writer
2,Actor
